In [2]:
import pandas as pd
from tqdm import tqdm

Lectura de archivos CSV

In [3]:
# Lista de rutas de los archivos CSV
Listarutas = [
    '../datos/base/datos-2013.csv', '../datos/base/datos-2014.csv', '../datos/base/datos-2015.csv',
    '../datos/base/datos-2016.csv', '../datos/base/datos-2017.csv', '../datos/base/datos-2018.csv',
    '../datos/base/datos-2019.csv', '../datos/base/datos-2020.csv', '../datos/base/datos-2021.csv'
]

# Cargar cada archivo en un DataFrame y ajustar el delimitador
listadfs = []
for ruta in tqdm(Listarutas):
    try:
        df = pd.read_csv(ruta, delimiter=';', on_bad_lines='skip')
        listadfs.append(df)
    except:
        print(f"Error al cargar {ruta}")


100%|██████████| 9/9 [00:04<00:00,  2.19it/s]


Unir dfs y eliminar nulos y duplicados

In [4]:
# Unir todos los DataFrames
df_unido = pd.concat(listadfs, ignore_index=True)

In [5]:
df_unido.shape 

(1026299, 16)

In [6]:
# Comprobacion valores nulos
valores_nulos = df_unido.isnull().sum()
print(valores_nulos)

CÓDIGO ÓRGÃO SUPERIOR         30359
NOME ÓRGÃO SUPERIOR          359206
CÓDIGO ÓRGÃO                  25117
NOME ÓRGÃO                    34887
CÓDIGO UNIDADE GESTORA        33581
NOME UNIDADE GESTORA          19481
CATEGORIA ECONÔMICA           18978
ORIGEM RECEITA                38418
ESPÉCIE RECEITA               31927
DETALHAMENTO                  29337
VALOR PREVISTO ATUALIZADO     51315
VALOR LANÇADO                 26419
VALOR REALIZADO               39465
PERCENTUAL REALIZADO          24134
DATA LANÇAMENTO               23831
ANO EXERCÍCIO                256574
dtype: int64


In [7]:
# Filtrar columnas "CÓDIGO ÓRGÃO" y "NOME ÓRGÃO", eliminando nulos y duplicados
orgao_df = df_unido[['CÓDIGO ÓRGÃO', 'NOME ÓRGÃO']].dropna().drop_duplicates()

# Crear y ordenar el diccionario por "CÓDIGO ÓRGÃO"
orgao_dict = dict(sorted(zip(orgao_df['CÓDIGO ÓRGÃO'], orgao_df['NOME ÓRGÃO'])))

In [8]:
# Creacion de diccionario inverso
orgao_inverso_dict = {v: k for k, v in orgao_dict.items()}

In [9]:
# Rellenar los valores nulos en base al diccionario creado
df_unido['NOME ÓRGÃO'] = df_unido.apply(
    lambda row: orgao_dict.get(row['CÓDIGO ÓRGÃO'], row['NOME ÓRGÃO'])
    if pd.isnull(row['NOME ÓRGÃO']) else row['NOME ÓRGÃO'],
    axis=1
)
df_unido['CÓDIGO ÓRGÃO'] = df_unido.apply(
    lambda row: orgao_inverso_dict.get(row['NOME ÓRGÃO'], row['CÓDIGO ÓRGÃO'])
    if pd.isnull(row['CÓDIGO ÓRGÃO']) else row['CÓDIGO ÓRGÃO'],
    axis=1
)

In [10]:
# Filtrar columnas "CÓDIGO ÓRGÃO SUPERIOR" y "NOME ÓRGÃO SUPERIOR", eliminando nulos y duplicados
orgao_superior_df = df_unido[['CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR']].dropna().drop_duplicates()

# Crear y ordenar el diccionario por "CÓDIGO ÓRGÃO SUPERIOR"
orgao_superior_dict = dict(sorted(zip(orgao_superior_df['CÓDIGO ÓRGÃO SUPERIOR'], orgao_superior_df['NOME ÓRGÃO SUPERIOR'])))

In [11]:
# Creacion de diccionario inverso
orgao_superior_inverso_dict = {v: k for k, v in orgao_superior_dict.items()}

In [12]:
# Rellenar los valores nulos en base al diccionario creado
df_unido['NOME ÓRGÃO SUPERIOR'] = df_unido.apply(
    lambda row: orgao_superior_dict.get(row['CÓDIGO ÓRGÃO SUPERIOR'], row['NOME ÓRGÃO SUPERIOR'])
    if pd.isnull(row['NOME ÓRGÃO SUPERIOR']) else row['NOME ÓRGÃO SUPERIOR'],
    axis=1
)
df_unido['CÓDIGO ÓRGÃO SUPERIOR'] = df_unido.apply(
    lambda row: orgao_superior_inverso_dict.get(row['NOME ÓRGÃO SUPERIOR'], row['CÓDIGO ÓRGÃO SUPERIOR'])
    if pd.isnull(row['CÓDIGO ÓRGÃO SUPERIOR']) else row['CÓDIGO ÓRGÃO SUPERIOR'],
    axis=1
)

In [13]:
# Filtrar columnas "CÓDIGO ÓRGÃO SUPERIOR" y "NOME ÓRGÃO SUPERIOR", eliminando nulos y duplicados
unidadegestora_df = df_unido[['CÓDIGO UNIDADE GESTORA', 'NOME UNIDADE GESTORA']].dropna().drop_duplicates()

# Crear y ordenar el diccionario por "CÓDIGO ÓRGÃO SUPERIOR"
unidadegestora_dict = dict(sorted(zip(unidadegestora_df['CÓDIGO UNIDADE GESTORA'], unidadegestora_df['NOME UNIDADE GESTORA'])))

In [14]:
# Creacion de diccionario inverso
unidadegestora_inverso_dict = {v: k for k, v in unidadegestora_dict.items()}


In [15]:
# Rellenar los valores nulos en base al diccionario creado
df_unido['NOME UNIDADE GESTORA'] = df_unido.apply(
    lambda row: unidadegestora_dict.get(row['CÓDIGO UNIDADE GESTORA'], row['NOME UNIDADE GESTORA'])
    if pd.isnull(row['NOME UNIDADE GESTORA']) else row['NOME UNIDADE GESTORA'],
    axis=1
)
df_unido['CÓDIGO UNIDADE GESTORA'] = df_unido.apply(
    lambda row: unidadegestora_inverso_dict.get(row['NOME UNIDADE GESTORA'], row['CÓDIGO UNIDADE GESTORA'])
    if pd.isnull(row['CÓDIGO UNIDADE GESTORA']) else row['CÓDIGO UNIDADE GESTORA'],
    axis=1
)

In [16]:
# Cambiar la columna 'DATA LANÇAMENTO' a formato de fecha
df_unido['DATA LANÇAMENTO'] = pd.to_datetime(df_unido['DATA LANÇAMENTO'], errors='coerce')

# Rellenar los valores nulos en 'ANO EXERCÍCIO' con el año de 'DATA LANÇAMENTO'
df_unido['ANO EXERCÍCIO'] = df_unido['ANO EXERCÍCIO'].fillna(df_unido['DATA LANÇAMENTO'].dt.year)

# Rellenar cualquier nulo restante en 'ANO EXERCÍCIO' con un 0
df_unido['ANO EXERCÍCIO'] = df_unido['ANO EXERCÍCIO'].fillna(0).astype(int)

# Borrar los datos con valor 0 an 'ANO EXERCICIO'
df_unido = df_unido[df_unido['ANO EXERCÍCIO'] != 0]

C:\Users\alexc\AppData\Local\Temp\ipykernel_19696\580477305.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_unido['DATA LANÇAMENTO'] = pd.to_datetime(df_unido['DATA LANÇAMENTO'], errors='coerce')


In [17]:
df_unido['ANO EXERCÍCIO'].unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [18]:
valores_nulos = df_unido.isnull().sum()
print(valores_nulos)

CÓDIGO ÓRGÃO SUPERIOR        10482
NOME ÓRGÃO SUPERIOR          10482
CÓDIGO ÓRGÃO                   900
NOME ÓRGÃO                     900
CÓDIGO UNIDADE GESTORA         608
NOME UNIDADE GESTORA           608
CATEGORIA ECONÔMICA          18876
ORIGEM RECEITA               38226
ESPÉCIE RECEITA              31721
DETALHAMENTO                 29174
VALOR PREVISTO ATUALIZADO    51026
VALOR LANÇADO                26272
VALOR REALIZADO              39216
PERCENTUAL REALIZADO         23997
DATA LANÇAMENTO              17902
ANO EXERCÍCIO                    0
dtype: int64


In [19]:
# Contar duplicados y nulos antes de eliminarlos
duplicados_antes = df_unido.duplicated().sum()
nulos_antes = df_unido.isnull().sum().sum()

# Eliminar duplicados y valores nulos
df_unido.drop_duplicates(inplace=True)
df_unido.dropna(inplace=True)

# Contar duplicados y nulos después de la eliminación
duplicados_despues = df_unido.duplicated().sum()
nulos_despues = df_unido.isnull().sum().sum()

# Calcular la diferencia para ver cuántos se eliminaron
duplicados_eliminados = duplicados_antes - duplicados_despues
nulos_eliminados = nulos_antes - nulos_despues

print(f"Duplicados eliminados: {duplicados_eliminados}")
print(f"Nulos eliminados: {nulos_eliminados}")

Duplicados eliminados: 108
Nulos eliminados: 300390


In [20]:
valores_nulos = df_unido.isnull().sum()
print(valores_nulos)

CÓDIGO ÓRGÃO SUPERIOR        0
NOME ÓRGÃO SUPERIOR          0
CÓDIGO ÓRGÃO                 0
NOME ÓRGÃO                   0
CÓDIGO UNIDADE GESTORA       0
NOME UNIDADE GESTORA         0
CATEGORIA ECONÔMICA          0
ORIGEM RECEITA               0
ESPÉCIE RECEITA              0
DETALHAMENTO                 0
VALOR PREVISTO ATUALIZADO    0
VALOR LANÇADO                0
VALOR REALIZADO              0
PERCENTUAL REALIZADO         0
DATA LANÇAMENTO              0
ANO EXERCÍCIO                0
dtype: int64


In [21]:
df_unido.shape

(765639, 16)

Traducir columnas del df unido

In [24]:
# Traducción de los nombres de las columnas al español
columnas_traducidas = {
    'CÓDIGO ÓRGÃO SUPERIOR': 'CODIGO_ORGANO_SUPERIOR',
    'NOME ÓRGÃO SUPERIOR': 'NOMBRE_ORGANO_SUPERIOR',
    'CÓDIGO ÓRGÃO': 'CODIGO_ORGANO',
    'NOME ÓRGÃO': 'NOMBRE_ORGANO',
    'CÓDIGO UNIDADE GESTORA': 'CODIGO_UNIDAD_GESTORA',
    'NOME UNIDADE GESTORA': 'NOMBRE_UNIDAD_GESTORA',
    'CATEGORIA ECONÔMICA': 'CATEGORIA_ECONOMICA',
    'ORIGEM RECEITA': 'ORIGEN_INGRESO',
    'ESPÉCIE RECEITA': 'TIPO_INGRESO',
    'DETALHAMENTO': 'DETALLE',
    'VALOR PREVISTO ATUALIZADO': 'VALOR_PREVISTO_ACTUALIZADO',
    'VALOR LANÇADO': 'VALOR_LANZADO',
    'VALOR REALIZADO': 'VALOR_REALIZADO',
    'PERCENTUAL REALIZADO': 'PORCENTAJE_REALIZADO',
    'DATA LANÇAMENTO': 'FECHA_LANZAMIENTO',
    'ANO EXERCÍCIO': 'AÑO_EJERCICIO'
}

df_unido.rename(columns=columnas_traducidas, inplace=True)

Guardar csv unido

In [23]:
# Guardar el DataFrame final en un archivo CSV
df_unido.to_csv('../datos/result/datos_brasil_unificado.csv', index=False)

In [27]:
# Calcular los índices de corte
split_index1 = len(df_unido) // 3
split_index2 = 2 * len(df_unido) // 3

# Dividir en tres partes
df1 = df_unido.iloc[:split_index1]
df2 = df_unido.iloc[split_index1:split_index2]
df3 = df_unido.iloc[split_index2:]

In [29]:
# Guardar 3 csv para subir a Git
df1.to_csv('../datos/result/datos_brasil_unificado_1.csv', index=False)
df2.to_csv('../datos/result/datos_brasil_unificado_2.csv', index=False)
df3.to_csv('../datos/result/datos_brasil_unificado_3.csv', index=False)